In [43]:
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Embedding
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.merge import Concatenate
from keras import backend as K
from w2v import train_word2vec 
import numpy as np
import difflib
import matplotlib.pyplot as plt

# Data Loading and Preprocessing

### Phrase -> index

In [42]:
# Read dictionary
phr_to_ind = dict()

with open('../../Datasets/SST1_dataset/dictionary.txt') as f:
    for line in f:
        entry = line.split('|')
        phr_to_ind[entry[0]] = int(entry[1])

print(len(phr_to_ind), phr_to_ind['Good'])

vocabulary_inv = dict((v, k) for k, v in phr_to_ind.items())
vocabulary_inv[0] = "<PAD/>"

keys = phr_to_ind.keys();

239232 14058


### Getting Index corresponding to sentences

In [50]:
# Loading data sentences
sentence_list = []
sentiment = []

with open('../../Datasets/SST1_dataset/datasetSentences.txt') as f:
    f.readline()
    for line in f:
        entry = line.split('\t')
        sent = entry[1][:-1]
        sent = sent.replace('Ã©', 'é')
        sent = sent.replace('Ã­', 'í')
        sent = sent.replace('-LRB-', '(')
        sent = sent.replace('-RRB-', ')')
        sent = sent.replace('Ã¼', 'ü')
        sent = sent.replace('Ã³', 'ó') 
        sent = sent.replace('Ã¡', 'á')
        sent = sent.replace('Ã¦', 'æ')
        sent = sent.replace('Ã¦', 'æ')
    
        if sent in phr_to_ind.keys():
            sentiment.append(phr_to_ind[sent])
        else:
            print('.', end="")
            keys_subset = [k for k in keys if (k[0] == sent[0])]
            key = difflib.get_close_matches(sent, keys_subset, n=1);
            sent = key[0]
            sentiment.append(phr_to_ind[sent])
            
        sentence_list.append(sent)
        
print(len(sentence_list))

......................11855


### Phrase Index -> Sentiment

In [58]:
ind_to_senti = dict()

with open('../../Datasets/SST1_dataset/sentiment_labels.txt') as f:
    f.readline()
    for line in f:
        entry = line.split('|')
        ind_to_senti[int(entry[0])] = float(entry[1])

print(len(ind_to_senti))

239232


### Assigning label to sentences

In [62]:
N_sent = len(sentence_list);
y_label = np.zeros((N_sent,), np.uint8)

counter = 0
for ind in sentiment:
    val = ind_to_senti[ind]
    if val >= 0.0 and val <= 0.2:
        y_label[counter] = 0;
    elif val > 0.2 and val <= 0.4:
        y_label[counter] = 1
    elif val > 0.4 and val <= 0.6:
        y_label[counter] = 2
    else:
        y_label[counter] = 3
    counter += 1

print(np.sum(y_label == 0), np.sum(y_label == 1), np.sum(y_label == 2), np.sum(y_label == 3))

1510 3140 2242 4963


### Loading train, test and valid split info

In [5]:
# Loading data split information

split_ind = []
with open('../../Datasets/SST1_dataset/datasetSplit.txt') as f:
    f.readline()
    for line in f:
        entry = line.split(',')
        split_ind.append(int(entry[1]))

print(len(split_ind))

N_train = split_ind.count(1)
N_test = split_ind.count(2)
N_valid = split_ind.count(3)
print (N_train, N_test, N_valid)

11855


In [42]:
x_train = np.zeros((N_train, max_sent_len), np.uint16)
x_test = np.zeros((N_test, max_sent_len), np.uint16)
x_valid = np.zeros((N_valid, max_sent_len), np.uint16)

c1, c2, c3 = 0,0,0
for i in range(len(x_all)):
    vec = x_all[i]
    if split_ind[i] == 1:
        x_train[c1,0:len(vec)] = np.uint16(vec)
        c1 += 1
    elif split_ind[i] == 2:
        x_test[c2,0:len(vec)] = np.uint16(vec)
        c2 += 1
    else:
        x_valid[c3,0:len(vec)] = np.uint16(vec)
        c3 += 1

# Training model 

In [49]:
embedding_dim = 300
min_word_count = 1
model_type = "CNN-static"
context = 10
embedding_weights = 

print("Model type is", model_type)
if model_type == "CNN-non-static" or model_type == "CNN-static":
    embedding_weights = train_word2vec(np.vstack((x_train, x_test)), vocabulary_inv, num_features=embedding_dim,
                                       min_word_count=min_word_count, context=context)
    if model_type == "CNN-static":
        x_train = embedding_weights[0][x_train]
        x_test = embedding_weights[0][x_test]
        
elif model_type == "CNN-rand":
    embedding_weights = None
    
else:
    raise ValueError("Unknown model type")

Model type is CNN-static
Load existing Word2Vec model '300features_1minwords_10context'


IndexError: arrays used as indices must be of integer (or boolean) type

In [54]:
embedding_weights[0].shape

(239232, 300)